In [11]:
import requests
url = 'http://127.0.0.1:5000/km/sorocaba/guarulhos'
req = requests.get(url)

print(req.json())

{'distance': 111.51, 'pedagios': 20.8}


In [11]:
from geopy.geocoders import Nominatim

from geopy.geocoders import Nominatim
def obter_latitude_longitude(cidade):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(cidade)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

def criar_string_pontos_por_cidades(cidades):
    coordenadas = []
    for cidade in cidades:
        latitude, longitude = obter_latitude_longitude(cidade)
        if latitude is not None and longitude is not None:
            coordenadas.append((latitude, longitude))
    pontos = ""
    for coordenada in coordenadas:
        pontos += f"{coordenada[1]},{coordenada[0]};"
    return pontos[:-1]  # Remove o último ponto e vírgula desnecessários


In [33]:
cidades = ["Jundiai", "Florianopolis"]
pontos_formatados = criar_string_pontos_por_cidades(cidades)

url_ajustada = 'https://rotasbrasil.com.br/roterizador/buscaRota/?pontos=' + pontos_formatados +'&alternatives=true&eixos=2&veiculo=1&st=false&combustivel=&consumo=&evitarPedagio=false&recaptchaRespostaCliente=03AFcWeA7j9naH_X1GiW2hXsNijAAl6OD4BGA4Ivh1_OyCx6W_v0z9EU26aLcc5pBUAh7-4RtMF16tD2fXZD-2A2pjC5DkhCEjsvhAnAss5WiYyTJbVc9_s-ggLNMRBIEWu8ZQW1AfHPqCSbKXkQ4XZhCxFgAzYzmF8udZAfhNEI7T6L6Jvcc6vF2ONx-ax9pIJAxmTz3OAFjy4dWCaGCMD9qwb17Fmxm44JP7EiCHm3lRdd_RGGWLQpUagh5gRix6x2U6R0CPffZ1uBpOv3oeM1pSjz0Eoutl_9HUB0nVJQNHIj7huLHV7wpIVj0B2VY5LvuHhh94ysYnWLu9-IqhLihEgOIVlgfjoCae5p0Yl2cUcmwHyyakjjbKpavnD2jfbkPPA97YQ2wcLBOFV6OqtkZ-dceO1yMIo1pp-PEXH7gz2j0QIFB-c8H158UzpacH-27tf8N1Ithk-88ckLZM3U2a1uAgQUhxYDm4Db3ZRFvyT_wydTfFZBGPgSTV1f2-ahy88wEhzxjPkeY7nTo1UNMZcrjnmiau9g--83jPyQSvlUi31MwUobR8AIHRkysOqMctf6WGtwZl9zSCPfurXU-9f95OTK9BZMnTxHpLl_G_7JwZAVc4bdDwh5BSGxPRKnBbeFdaus462FQckgFrtH3aH6Bb4fA_wBIKa5umFMy3xSbXhIXmumPpzUCtXLEyEl30cbrdConCE9_BvoMXwsXqD52bRqONT7rH_KzthaTT0DgE1Eb9x5U&recaptchaRespostaV3=true&evitarBalsa=false&meioPagamento=&fornecedorPagamento=&dataTarifa=2024-02-29'

In [34]:
import requests

def chamar_api_rotas_brasil(url):
    headers = {
        'authority': 'rotasbrasil.com.br',
        'accept': '*/*',
        'accept-language': 'pt-BR,pt;q=0.7',
        'cookie': 'RBNOTIFICATION=true; RBCOUNTNOTIFICATION=19; JSESSIONID=DD991B79A608CF2F40C3DDD0428DD789; veiculo=1; eixo=2; precoCombustivel=; consumo=',
        'referer': 'https://rotasbrasil.com.br/',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Brave";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'sec-gpc': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest'
    }
    response = requests.get(url, headers=headers)
    return response.json()


resposta = chamar_api_rotas_brasil(url_ajustada)

distance = resposta['routes'][0]['distance']

print(distance)


725659.1


In [78]:
def procurar_chave(json_data, chave_alvo):
    resultados = []

    def buscar(json_data, chave_alvo, caminho=''):
        if isinstance(json_data, dict):
            for chave, valor in json_data.items():
                novo_caminho = f"{caminho}.{chave}" if caminho else chave
                if chave == chave_alvo:
                    resultados.append(novo_caminho)
                buscar(valor, chave_alvo, novo_caminho)
        elif isinstance(json_data, list):
            for index, item in enumerate(json_data):
                novo_caminho = f"{caminho}[{index}]"
                buscar(item, chave_alvo, novo_caminho)

    buscar(json_data, chave_alvo)
    return resultados

procurar_chave(resposta, 'pedagios')

pedagios = resposta['routes'][0]['pedagios']
import json

json_data = json.dumps(pedagios) 


In [79]:
lista_de_dados = json.loads(json_data)
print(lista_de_dados)

# Iterando sobre os valores da nona posição de cada sublista
def calcular_pedagio(loads):
    total_pedagio = 0
    for sublista in loads:
        # A nona posição de cada sublista contém o valor do pedágio
        total_pedagio += float(sublista[8])  # Somando os valores do pedágio
    return total_pedagio
    
ped = calcular_pedagio(lista_de_dados)   

print(ped)

[[128, '-23.347179179019072', '-46.813198364527466', 0, 'AUTOBAN', 'Caieiras', 'SP-348', '36,200', 12.4, 20924.31587808347, 1, None, 1, 'Todos os dias'], [56, '-23.600033629566326', '-46.81279040142181', 1, 'RODOANEL OESTE', 'Régis Bittencourt', 'SP-021', '25,360', 3.0, 54946.614214876776, 1, None, 1, 'Todos os dias'], [191, '-23.787651336323094', '-46.913884878158576', 1, 'RÉGIS BITTENCOURT', 'São Lourenço da Serra', 'BR-116', '298,800', 4.0, 79304.57986557484, 1, None, 1, 'Todos os dias'], [212, '-24.157849318625274', '-47.326239174274406', 1, 'RÉGIS BITTENCOURT', 'Miracatu', 'BR-116', '370,400', 4.0, 147496.1199157828, 1, None, 1, 'Todos os dias'], [362, '-24.389178800060517', '-47.72249747486876', 1, 'RÉGIS BITTENCOURT', 'Juquiá', 'BR-116', '426,600', 4.0, 202001.57429597765, 1, None, 1, 'Todos os dias'], [377, '-24.72242885434674', '-48.0825938687172', 1, 'RÉGIS BITTENCOURT', 'Cajati', 'BR-116', '485,700', 4.0, 259136.78774945968, 1, None, 1, 'Todos os dias'], [376, '-24.964063899

In [23]:


valor_distance = resposta['routes'][0]['routes'][0]['legs'][0]['distance']

print(valor_distance)


KeyError: 'routes'

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/helio/Documentos/GitHub/flask-busca-km/env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/helio/Documentos/GitHub/flask-busca-km/env/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/helio/Documentos/GitHub/flask-busca-km/env/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

SystemExit: 1

/home/helio/Documentos/GitHub/flask-busca-km/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [43]:
import requests

url = url_ajustada

payload = {}
headers = {
  'authority': 'rotasbrasil.com.br',
  'accept': '*/*',
  'accept-language': 'pt-BR,pt;q=0.7',
  'cookie': 'RBNOTIFICATION=true; RBCOUNTNOTIFICATION=19; JSESSIONID=DD991B79A608CF2F40C3DDD0428DD789; veiculo=1; eixo=2; precoCombustivel=; consumo=',
  'referer': 'https://rotasbrasil.com.br/',
  'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Brave";v="122"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Linux"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'sec-gpc': '1',
  'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
  'x-requested-with': 'XMLHttpRequest'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"routes":[{"duration":51193.8,"distance":1027601.9,"legs":[{"duration":35950.6,"summary":"Rodovia Régis Bittencourt, Rodovia Governador Mário Covas","distance":725659.1,"weight":35950.6,"steps":[{"mode":"driving","duration":13.9,"distance":177.3,"name":"Rua do Rosário","weight":13.9,"geometry":"vaplCxdd}Gb@WhCaB|AaA","driving_side":"right","intersections":[{"entry":[true],"bearings":[148],"location":[-46.884772,-23.188919],"out":0},{"entry":[true,false,false],"bearings":[150,240,330],"in":2,"location":[-46.884647,-23.189104],"out":0}],"maneuver":{"modifier":"left","bearing_after":148,"bearing_before":0,"location":[-46.884772,-23.188919],"type":"depart"}},{"mode":"driving","duration":20,"distance":193.5,"name":"Rua Engenheiro Monlevade","weight":20,"geometry":"bjplC|~c}G|AlDvArC","driving_side":"right","intersections":[{"entry":[false,true,true,false],"bearings":[60,150,240,330],"in":3,"location":[-46.883829,-23.190257],"out":2},{"entry":[false,false,true,true],"bearings":[60,150,240,3

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)


usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/home/helio/.local/share/jupyter/runtime/kernel-v2-627699Fd3NcVRzyHM.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'